In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [ ]:
def grayscale(img):
    #covert the image to gray scale
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def gaussian_blur(img, kernel_size):
    #perfoms gaussian blur
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

    
def canny(img, low_threshold, high_threshold):
    #performing canny edge detection
    return cv2.Canny(img, low_threshold, high_threshold)


def region_of_interest(img, vertices):

    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #Masking the matrix within the vertices with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):

    #performinh Hough Transform
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    color=[255, 0, 0]
    thickness=10
    
    #Selecting the lines to be drawn on the left side lane
    tmp=[]
    for line in lines:
        for x1,y1,x2,y2 in line:
            if(x1<460 and x2<460 and y1>320 and y2>320):
                tmp.append([x1,y1,x2,y2])
    
    x1=min(list(map(lambda x:x[0],tmp)))
    y1=max(list(map(lambda x:x[1],tmp)))
    x2=max(list(map(lambda x:x[2],tmp)))
    y2=min(list(map(lambda x:x[3],tmp)))
    
    a = np.array([[x1,1], [x2,1]])
    b = np.array([(540-y1),(540-y2)])
    c = np.linalg.solve(a, b)
    
    x1=int((c[1]/c[0])*-1)
    y1=540
    
    cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
    
    #Selecting the line to be drawn on right side of the lane
    tmp=[]
    for line in lines:
        for x1,y1,x2,y2 in line:
            if(x1>500 and x2>500 and y1>320 and y2>320):
                tmp.append([x1,y1,x2,y2])
                
    x1=min(list(map(lambda x:x[0],tmp)))
    y1=min(list(map(lambda x:x[1],tmp)))
    x2=max(list(map(lambda x:x[2],tmp)))
    y2=max(list(map(lambda x:x[3],tmp)))
    
    a = np.array([[x1,1], [x2,1]])
    b = np.array([(540-y1),(540-y2)])
    c = np.linalg.solve(a, b)

    x2=int((c[1]/c[0])*-1)
    y2=540
    
    cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
    
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):

    #returning the final image
    return cv2.addWeighted(initial_img, α, img, β, γ)


In [ ]:
#Variables initialized
kernel_size = 5
low_threshold = 50
high_threshold = 150
vertices = np.array([[(85,540),(460, 320), (500, 320), (900,540)]], dtype=np.int32)
rho = 2 
theta = np.pi/180 
threshold = 10    
min_line_length = 20
max_line_gap = 30

In [ ]:
#Function to call all the image frames of the video to perform lane detection
def process_image(image):
    img=grayscale(image)
    img=gaussian_blur(img,kernel_size)
    canny_img=canny(img,low_threshold,high_threshold)
    img=region_of_interest(canny_img,vertices)
    img=hough_lines(img, rho, theta, threshold, min_line_length, max_line_gap)
    img=weighted_img(img,image)
    return img

In [ ]:
#Exeuting the video and saving it to the output folder
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
#Exeuting the video and saving it to the output folder
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
images=os.listdir("test_images/")
for i in images:
    image = mpimg.imread('test_images/'+str(i))
    img = process_image(image)
    mpimg.imsave("test_images_ouput/"+str(i), img)